We start by downloading the necessary libraries

In [0]:
!git clone https://github.com/adines/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 398 (delta 0), reused 4 (delta 0), pack-reused 392
Receiving objects: 100% (398/398), 30.92 MiB | 20.19 MiB/s, done.
Resolving deltas: 100% (140/140), done.


Download the images.

In [0]:
!wget https://www.dropbox.com/s/30j6fugdwekgypd/unetData.zip?dl=0 -O unetData.zip
!unzip unetData.zip

--2019-01-28 11:29:48--  https://www.dropbox.com/s/30j6fugdwekgypd/unetData.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/30j6fugdwekgypd/unetData.zip [following]
--2019-01-28 11:29:48--  https://www.dropbox.com/s/raw/30j6fugdwekgypd/unetData.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc17bbeab93cf612afbc95369c73.dl.dropboxusercontent.com/cd/0/inline/AaRv4rSbzPPrRgnm1twKWJRhgQNICvTVOViPGb85uTVOcppbpUSs42mD_8qZHTotRvcpWLxFPpoc8xmkNZOiGzPa9MAPFhcN8BVtQu4V1S501w/file# [following]
--2019-01-28 11:29:48--  https://uc17bbeab93cf612afbc95369c73.dl.dropboxusercontent.com/cd/0/inline/AaRv4rSbzPPrRgnm1twKWJRhgQNICvTVOViPGb85uTVOcppbpUSs42mD_8qZHTotRvcpWLxFPpoc8xmkNZOiGzPa9MAPFhcN8BVtQu4V1S501w/file
Resolving u

Install some modules.

In [0]:
!pip install clodsa

    100% |████████████████████████████████| 1.5MB 17.4MB/s 
  Running setup.py bdist_wheel for clodsa ... - \ done
  Stored in directory: /root/.cache/pip/wheels/a7/e7/0b/8fc26527f6feeef05e8ed273730418ea147cf3223d65d0123a
  Running setup.py bdist_wheel for mahotas ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/fb/2a/81/12bc1070527dc81ef856f9d36db9ec0a42e8c14985d005395b
  Running setup.py bdist_wheel for commentjson ... - done
  Stored in directory: /root/.cache/pip/wheels/8c/0e/36/56d4e5ddf02e0ddd1d8f9eb8d90975a5192e0155ffe28ff19c
Successfully built clodsa mahotas commentjson


In [0]:
!pip install keras-metrics

Move to the unet folder.

In [0]:
cd unet/

/content/unet


Load the necessary modules.

In [0]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
from keras.optimizers import SGD
from clodsa.utils.minivgg import MiniVGGNet
import cv2
%matplotlib inline

from model import *
from data import *
import keras_metrics

Using TensorFlow backend.


Apply the augmentation process.

In [0]:
PROBLEM = "semantic_segmentation"
ANNOTATION_MODE = "folders"
INPUT_PATH = "/content/unetData/train/"
GENERATION_MODE = "linear"
OUTPUT_MODE = "folders"
OUTPUT_PATH= "/content/unetData/train/"
PARAMETERS = {
    "batchSize": 2,
    "width": 256,
    "height": 256,
    "labelsExtension":".png",
    "outputPath": OUTPUT_PATH
  }
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,PARAMETERS)
transformer = transformerGenerator(PROBLEM)

rotateRandom = createTechnique("rotate",{})
augmentor.addTransformer(transformer(rotateRandom))
for angle in [90,180,270]:
    rotate = createTechnique("rotate", {"angle" : angle})
    augmentor.addTransformer(transformer(rotate))

translation = createTechnique("translation", {"x":15,"y":-5})
augmentor.addTransformer(transformer(translation))

vFlip = createTechnique("flip",{"flip":0})
augmentor.addTransformer(transformer(vFlip))

hFlip = createTechnique("flip",{"flip":1})
augmentor.addTransformer(transformer(hFlip))

shear = createTechnique("shearing", {"a":0.5})
augmentor.addTransformer(transformer(shear))

eqHist = createTechnique("equalize_histogram",{})
augmentor.addTransformer(transformer(eqHist))

gamma = createTechnique("gamma",{"gamma":1.5})
augmentor.addTransformer(transformer(gamma))

elastic = createTechnique("elastic",{"alpha":5,"sigma":0.05})
augmentor.addTransformer(transformer(elastic))

augmentor.applyAugmentation()

In [0]:
print("Number of nuclei images")
!ls /content/unetData/train/images/ | wc -l
print("Number of masks for nuclei images")
!ls /content/unetData/train/labels/ | wc -l

Number of nuclei images
240
Number of masks for nuclei images
240


Train the model.

In [0]:
data_gen_args = dict()
myGene = trainGenerator(2,'/content/unetData/train','images','labels',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=1200,epochs=5,callbacks=[model_checkpoint])

/content/unet/model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Found 240 images belonging to 1 classes.
Found 240 images belonging to 1 classes.
Epoch 1/5
1200/1200 [==============================] - 521s 434ms/step - loss: 0.5263 - true_positive: 115090150.0000 - true_negative: 9562525.0000 - false_positive: 30126415.0000 - false_negative: 2507310.0000 - acc: 0.7925 - f1_score: 0.8758 - precision: 0.7925 - recall: 0.9787

Epoch 00001: loss improved from inf to 0.52629, saving model to unet_membrane.hdf5
Epoch 2/5
1200/1200 [==============================] - 522s 435ms/step - loss: 0.1907 - true_positive: 111555592.0000 - true_negative: 32271977.0000 - false_positive: 7416963.0000 - false_negative: 6041868.0000 - acc: 0.9144 - f1_score: 0.9431 - precision: 0.9377 - recall: 0.9486

Epoch 00002: loss improved from 0.52629 to 0.19072, saving model to unet_membrane.hdf5
Epoch 3/5
1200/1200 [==============================] - 522s 435ms/step - loss: 0.1541 - true_positive: 112549416.0000 - true_negative: 33676718.0000 - false_positive: 6012222.0000 - fa

Evaluate the model.

In [0]:
import cv2
import glob
import numpy as np
def geneTrainNpyPrueba(image_path,mask_path,flag_multi_class = False,num_class = 2,image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(image_path+"*")
    mask_name_arr = glob.glob(mask_path+"*")
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = cv2.imread(item,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(256,256))
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = cv2.imread(mask_name_arr[index],cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,(256,256))
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr

In [0]:
image_arr, mask_arr=geneTrainNpyPrueba('/content/unetData/test/images/','/content/unetData/test/labels/')
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.evaluate(image_arr,mask_arr)
print(results)

/content/unet/model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


10/10 [==============================] - 5s 524ms/step
[0.3251470923423767, 490056, 106578, 28687, 30039, 0.9103912115097046, 0.9434694803790409, 0.9446990128057738, 0.9422432440225454]


In [0]:
TP=results[1]
TN=results[2]
FP=results[3]
FN=results[4]

In [0]:
TNR=TN/(TN+FP)
print(TNR)

0.7879200088714745
